In [1]:
!pip install bs4
!pip install contractions
!pip install gensim
!pip install nltk
!pip install -U scikit-learn scipy matplotlib
!pip install torch

     ---------------------------------------- 8.4/8.4 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 4.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.0
    Uninstalling matplotlib-3.7.0:
      Successfully uninstalled matplotlib-3.7.0


In [3]:
import pandas as pd
import numpy as np
import re
import contractions
from collections import defaultdict

from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as data_utils

from bs4 import BeautifulSoup

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

pd.options.display.max_colwidth = 500

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jaehw\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# 1. Dataset Generation

## Read Data

In [4]:
df = pd.read_csv("amazon_reviews_us_Beauty_v1_00.tsv.gz", compression='gzip', header=0,sep='\t', quotechar='"', on_bad_lines='skip')

C:\Users\jaehw\AppData\Local\Temp\ipykernel_22240\4067534945.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("amazon_reviews_us_Beauty_v1_00.tsv.gz", compression='gzip', header=0,sep='\t', quotechar='"', on_bad_lines='skip')


## Keep Reviews and Ratings

In [5]:
parsed_df = df[["star_rating", "review_headline", "review_body"]]
parsed_df.dropna()

,star_rating,review_headline,review_body
0,5,Five Stars,"Love this, excellent sun block!!"
1,5,Thank you Alba Bontanica!,The great thing about this cream is that it doesn't smell weird like all those chemical laden ones. I get a nice healthy un-fake looking tan that isn't orange and it makes my skin soft too.
2,5,Five Stars,"Great Product, I'm 65 years old and this is all it claims to be!"
3,5,GOOD DEAL!,I use them as shower caps & conditioning caps. I like that they're in bulk. It saves a lot of money.
4,5,this soaks in quick and provides a nice base for makeup,"This is my go-to daily sunblock. It leaves no white cast at all and has a clean, pleasant scent. If you're a makeup wearer, this soaks in quick and provides a nice base for makeup. I've been using this brand for over a year. With daily use, this tube will last you a couple months."
...,...,...,...
5094302,5,Great Little Grooming Tool,"After watching my Dad struggle with his scissors to clip, what he affectionately calls his 'tuffs of ear hair'. I bought him this electric clippers...now we do we hear him mumble..and fuss about how he is certain he will cut off his ear someday.....This is a great invention...it moves at lightenind speed and clips those hairs neaty...... Great Price too!"
5094303,3,Not bad for the price,"Like most sound machines, the sounds choices are limited and most have a very noticeable cycle. The brook sound actually had a click at the end! However, the ocean and white noise had a good cycle.<br />Unfortunately, only after a year, it broke. Now, I'm looking for a better one with more sounds."
5094304,5,Best Curling Iron Ever,"I bought this product because it indicated 30 second heat up time. It is great. You plug it in, hit the on button, select a heat level (1-15) , and in less than 30 seconds it is hot. No more waiting around for the iron to heat up. Quick touch ups take no time at all. I'll never go back to the &quot;old style&quot; plug and wait."
5094305,5,"The best electric toothbrush ever, REALLY!",We have used Oral-B products for 15 years; this new model is even better. It is stronger yet thinner; generates different vibrations (3) around the toothbrush head and varies this according to pressure. Also has a built-in timer. Enjoy!


In [6]:
parsed_df.head()

,star_rating,review_headline,review_body
0,5,Five Stars,"Love this, excellent sun block!!"
1,5,Thank you Alba Bontanica!,The great thing about this cream is that it doesn't smell weird like all those chemical laden ones. I get a nice healthy un-fake looking tan that isn't orange and it makes my skin soft too.
2,5,Five Stars,"Great Product, I'm 65 years old and this is all it claims to be!"
3,5,GOOD DEAL!,I use them as shower caps & conditioning caps. I like that they're in bulk. It saves a lot of money.
4,5,this soaks in quick and provides a nice base for makeup,"This is my go-to daily sunblock. It leaves no white cast at all and has a clean, pleasant scent. If you're a makeup wearer, this soaks in quick and provides a nice base for makeup. I've been using this brand for over a year. With daily use, this tube will last you a couple months."


In [7]:
del(df)

 ## We form three classes and select 20000 reviews randomly from each class.

In [8]:
class1_df = parsed_df.loc[parsed_df['star_rating'].isin([1,2])].sample(20000)
class2_df = parsed_df.loc[parsed_df['star_rating'] == 3].sample(20000)
class3_df = parsed_df.loc[parsed_df['star_rating'].isin([4,5])].sample(20000)

In [9]:
class1_df["class"] = 1
class2_df["class"] = 2
class3_df["class"] = 3

In [10]:
final_df = pd.concat([class1_df, class2_df, class3_df])

final_df['review_headline'] = final_df['review_headline'].apply(str)
final_df['review_body'] = final_df['review_body'].apply(str)

In [11]:
final_df['review'] = final_df[['review_headline', 'review_body']].agg(' '.join, axis=1)

In [12]:
final_df = final_df.drop('star_rating', axis=1)
final_df = final_df.drop('review_headline', axis=1)
final_df = final_df.drop('review_body', axis=1)

In [13]:
final_df[final_df['class'] == 1].size

40000

In [14]:
del(class1_df)
del(class2_df)
del(class3_df)

# Data Cleaning



# Pre-processing

In [15]:
# Lowercasing
final_df["review"] = final_df["review"].str.lower()

In [16]:
def getRidOfNonAlphabet(s):
  return re.sub(r"[^a-zA-Z]+", ' ', s)

def getRidOfHTML(s):
  return BeautifulSoup(s, "lxml").text

def getRidOfURL(s):
  return re.sub(r'http\S+', '', s)

def contractions(s):
  # Also gets rid of extra spaces
  import contractions
  ans = []
  for word in s.split():
    ans.append(contractions.fix(word))
  return ' '.join(ans)

## Pre-processing by getting rid of non-alphabet characters, any HTML/URLs, and contractions

In [17]:
final_df["review"] = final_df["review"].apply(getRidOfURL).apply(getRidOfHTML).apply(contractions).apply(getRidOfNonAlphabet)

c:\Users\jaehw\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


## Remove the stop words 

In [18]:
def getRidOfStopWords(s):
  stopWords = set(stopwords.words('english'))
  words = word_tokenize(s)
  filteredWords = []
  for word in words:
      if word not in stopWords:
          filteredWords.append(word)
  return ' '.join(filteredWords)

In [19]:
final_df["review"] = final_df["review"].apply(getRidOfStopWords)

## perform lemmatization & stemming

In [20]:
from nltk.stem import PorterStemmer 

def lemmatize(s):
  lemmatizer = WordNetLemmatizer()
  words = word_tokenize(s)
  filteredWords = []
  for word in words:
      filteredWords.append(lemmatizer.lemmatize(word))
  return ' '.join(filteredWords)

def stem(s):
    filteredWords = []
    ps = PorterStemmer()
    for word in s.split():
        filteredWords.append(ps.stem(word))
    return ' '.join(filteredWords)

In [21]:
final_df["formatted_review"] = final_df["review"].apply(lemmatize).apply(stem)

In [22]:
stem("heat hot sampled heat buying loved arrived smelled old like experienced previously")

'heat hot sampl heat buy love arriv smell old like experienc previous'

## testing_df might be used to train our custom Word2Vec

In [23]:
# testing_df = parsed_df

# testing_df['review_headline'] = testing_df['review_headline'].apply(str)
# testing_df['review_body'] = testing_df['review_body'].apply(str)

# testing_df['review'] = testing_df[['review_headline', 'review_body']].agg(' '.join, axis=1)

# testing_df["review"] = testing_df["review"].apply(getRidOfURL).apply(getRidOfHTML).apply(contractions).apply(getRidOfNonAlphabet)

c:\Users\jaehw\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


KeyboardInterrupt: 

# 2. Word Embedding (25 points)

##  a) word2vec-google-news-300

In [23]:
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

In [24]:
# # Not my example, checking if King − Man + Woman = Queen
# predKing = wv.most_similar(positive=["man","queen"], negative=["woman"], topn=1)
# print(predKing)

[('king', 0.6958590149879456)]


In [25]:
# # Still not my example
# predOutstanding = wv.most_similar(positive=["excellent"], topn=20)
# print(predOutstanding)

# wv.similarity(w1="excellent", w2="outstanding")

[('terrific', 0.7409726977348328), ('superb', 0.7062715888023376), ('exceptional', 0.681470513343811), ('fantastic', 0.6802847981452942), ('good', 0.6442928910255432), ('great', 0.6124600172042847), ('Excellent', 0.6091997623443604), ('impeccable', 0.5980967283248901), ('exemplary', 0.5959650278091431), ('marvelous', 0.582928478717804), ('ideal', 0.5744982361793518), ('impressive', 0.5732302069664001), ('phenomenal', 0.5656529664993286), ('incredible', 0.563998281955719), ('wonderful', 0.5614795684814453), ('decent', 0.5584126114845276), ('outstanding', 0.556748628616333), ('solid', 0.5502715110778809), ('amazing', 0.5470210909843445), ('perfect', 0.540958821773529)]


0.55674857

In [26]:
# ## Example 1 winter - cold + hot = summer
# predWinter = wv.most_similar(positive=["cold","summer"], negative=["hot"], topn=1)
# print(predWinter)

[('winter', 0.6970129013061523)]


In [27]:
# ## Example 2 italian - china + chinese = italy
# predAnt = wv.most_similar(positive=["italy","chinese"], negative=["china"], topn=10)
# print(predAnt)

[('italians', 0.5945354700088501), ('italian', 0.5859934687614441), ('barcelona', 0.5842419862747192), ('juve', 0.5681446194648743), ('malta', 0.564256489276886), ('spain', 0.5633350014686584), ('montreal', 0.5586921572685242), ('ac_milan', 0.5585262775421143), ('diego', 0.5546826720237732), ('real_madrid', 0.5530885457992554)]


In [28]:
# ## Example 3 smaller - big + small = bigger
# predAnt = wv.most_similar(positive=["smaller","big"], negative=["small"], topn=10)
# print(predAnt)

[('bigger', 0.7836998701095581), ('larger', 0.5866796970367432), ('Bigger', 0.5707237720489502), ('biggest', 0.5240510702133179), ('splashier', 0.5107756853103638), ('huge', 0.5084124803543091), ('sharper', 0.48242926597595215), ('heavier', 0.4784497320652008), ('tougher', 0.4632209837436676), ('better', 0.4631742835044861)]


In [29]:
# score = wv.evaluate_word_analogies("questions-words.txt", case_insensitive=True)[0]
# print(str(score*100) + "% Accuracy for Google's Word2Vec")

74.01448525607863% Accuracy for Google's Word2Vec


## b) Custom Word2Vec

In [30]:
import gensim.models

In [31]:
params = {
    'vector_size': 300,
    'window': 13,
    'min_count': 9,
}

In [32]:
# sent = [e.split() for e in final_df.review.to_list()]
sent = [e.split() for e in testing_df.review.to_list()]

In [33]:
model = gensim.models.Word2Vec(sent, **params)

KeyboardInterrupt: 

In [ ]:
# Not my example, checking if King − Man + Woman = Queen
predKing = model.wv.most_similar(positive=["man","queen"], negative=["woman"], topn=5)
print(predKing)

: 

In [ ]:
predKing = wv2.wv.most_similar(positive=["man","queen"], negative=["woman"], topn=10)
print(predKing)

: 

In [ ]:
## Example 1 winter - cold + hot = summer
predWinter = wv2.wv.most_similar(positive=["cold","summer"], negative=["hot"], topn=10)
print(predWinter)

: 

In [ ]:
## Example 2 italian - china + chinese = italy
predAnt = wv2.wv.most_similar(positive=["italy","chinese"], negative=["china"], topn=10)
print(predAnt)

: 

In [ ]:
## Example 3 smaller - big + small = bigger
predAnt = wv.most_similar(positive=["smaller","big"], negative=["small"], topn=10)
print(predAnt)

: 

In [ ]:
score = wv2.wv.evaluate_word_analogies("questions-words.txt", case_insensitive=True)[0]
print(str(score*100) + "% Accuracy for My Word2Vec")

: 

Comparing my custom Word2Vec with Google's Word2Vec, it's clear that Google's Word2Vec is better. Although I am able to pass some examples, such as small-smaller = big-bigger like Google has, I'm not able to pass many other examples. The way I was able to pass some examples myself was by cleaning up the dataset a bit more. What I've also done was I used all of the reviews that exist in the Amazon dataset to give my Word2Vec more data, and that didn't seem to help it as much as cleaning up the dataset. In fact, increasing the size of my dataset to train my custom Word2Vec gave it an overall lower score in questions-words.txt, from 12% -> 7%. 

I've also utilized questions-words.txt to measure the accuracy of both Word2Vec models. This text file has load of corresponding correlated words such as 
"Athens Greece = Baghdad Iraq"
all categorized by countries, food, etc. Google's Word2Vec score is higher than that of my model's as shown above, proving that Google's model encodes better rsemantic similiarites between words than my model.

https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt

# 3) Simple models (20 points)

## TF-IDF/Word2VecFeature Extraction

In [ ]:
# Average Word2Vec Vectors
def vectorizeReview(text):
    text_vector = []
    for word in text.split():
        try:
            text_vector.append(wv[word])
        except KeyError:
            pass
    if not text_vector:
        return np.zeros(300)
    return np.mean(text_vector, axis=0)

: 

In [ ]:
# First 10 Word2Vec Vectors
def vectorizeReview10(text):
    text_vector = []
    i = 0
    for ind, word in enumerate(text.split()):
        try:
            text_vector.append(wv2.wv[word])
            i += 1
        except KeyError:
            pass
        if i >= 10:
            break
            
    while i < 10:
        text_vector.append(np.zeros(300))
        i += 1
    return text_vector

: 

In [ ]:
# First 20 Word2Vec Vectors
def vectorizeReview20(text):
    text_vector = []
    i = 0
    for ind, word in enumerate(text.split()):
        try:
            text_vector.append(wv2.wv[word])
            i += 1
        except KeyError:
            pass
        if i >= 20:
            break
            
    while i < 20:
        text_vector.append(np.zeros(300))
        i += 1
    return text_vector

: 

In [ ]:
final_df["vector_review"] = final_df["formatted_review"].apply(vectorizeReview)

: 

In [ ]:
final_df["vector_review_10"] = final_df["formatted_review"].apply(vectorizeReview10)

: 

In [ ]:
final_df["vector_review_20"] = final_df["formatted_review"].apply(vectorizeReview10)

: 

In [ ]:
final_df.reset_index(drop=True, inplace=True)

: 

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(final_df['vector_review'], final_df['class'], test_size=0.2, random_state=42)
train_fx, test_fx, train_fy, test_fy = train_test_split(final_df['formatted_review'], final_df['class'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=300)
vectorizer.fit(final_df['formatted_review'])
train_tfidf = vectorizer.transform(train_fx)
test_tfidf = vectorizer.transform(test_fx)

Encoder = LabelEncoder()
train_result = Encoder.fit_transform(train_y)
test_result = Encoder.fit_transform(test_y)

: 

## SVM

In [ ]:
model = svm.LinearSVC()
model.fit(list(train_x), train_result)

result = model.predict(list(test_x))
acc = accuracy_score(test_result, result)
print(str(acc * 100) + "& Accuracy for SVM w/ Word2Vec")

: 

In [ ]:
model = svm.LinearSVC()
train_tfidf = vectorizer.transform(train_fx)
model.fit(train_tfidf, train_result)

result = model.predict(test_tfidf)
acc = accuracy_score(test_result, result)
print(str(acc * 100) + "& Accuracy for SVM w/ TFIDF")

: 

## Perceptron

In [ ]:
perceptron = Perceptron()
perceptron.fit(list(train_x), train_result)

result = perceptron.predict(list(test_x))
acc = accuracy_score(test_result, result)
print(str(acc * 100) + "& Accuracy for Perceptron w/ Word2Vec")

: 

In [ ]:
perceptron = Perceptron()
perceptron.fit(train_tfidf, train_result)

result = perceptron.predict(test_tfidf)
acc = accuracy_score(test_result, result)
print(str(acc * 100) + "& Accuracy for Perceptron w/ TFIDF")

: 

TF-IDF and Word2Vec are both NLP techinques, and initial thought I had in mind was that Word2Vec, which captures relationship between words, will give a better score than TF-IDF. But when testing the both techniques, I was unable to consistently get Word2Vec to be better than TF-IDF. Perceptron was able to get higher score commonly, but I was not able to get SVM to get higher score.

My assumption in why this might be the case is that TF-IDF is better at capturing important keywords (and phrases). This might give Perceptron something to pay attention to in order to give it better class.  

# 4. Feedforward Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

: 

In [ ]:
num_train = len(final_df)
indices = list(range(num_train))
np.random.shuffle(indices)
valid_size = 0.2
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

: 

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

: 

## a) Average Word2Vec

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
valid = data_utils.TensorDataset(torch.Tensor(np.vstack(final_df["vector_review"])), torch.Tensor(e))
valid_loader = data_utils.DataLoader(valid, batch_size = 20, shuffle = True)

: 

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
train = data_utils.TensorDataset(torch.Tensor(np.vstack(final_df["vector_review"])), torch.Tensor(e))
train_loader = data_utils.DataLoader(train, batch_size = 20, shuffle = True)

: 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(300, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

: 

In [ ]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 10

best_accuracy = float('-inf')
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    model.eval() 
    correct = 0
    for data, target in valid_loader:
        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)

        e = torch.argmax(output, dim=1)
        correct += (e == target).float().sum()

    accuracy = correct.item()/len(valid_loader.dataset) * 100 
    
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'perceptron_avg.pt')
    print("Epoch " + str(epoch) + " - " + str(accuracy) +"% Accuracy")
print("Best Accuracy for Perceptron model w/ avg vectors: " + str(best_accuracy) + "%")

: 

## b) First 10 Word2Vec Vectors

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
valid = data_utils.TensorDataset(torch.Tensor(final_df["vector_review_10"]), torch.Tensor(e))
valid_loader = data_utils.DataLoader(valid, batch_size = 20, shuffle = True)

: 

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
train = data_utils.TensorDataset(torch.Tensor(final_df["vector_review_10"]), torch.Tensor(e))
train_loader = data_utils.DataLoader(train, batch_size = 20, shuffle = True)

: 

In [ ]:
class PerceptronL(nn.Module):
    def __init__(self):
        super(PerceptronL, self).__init__()
        self.fc1 = nn.Linear(300*10, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = x.view(-1, 300*10) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

: 

In [ ]:
model = PerceptronL()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 10

best_accuracy = float('-inf')
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    model.eval() 
    correct = 0
    for data, target in valid_loader:
        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)

        e = torch.argmax(output, dim=1)
        correct += (e == target).float().sum()

    accuracy = correct.item()/len(valid_loader.dataset) * 100 
    
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'perceptron_list.pt')
    print("Epoch " + str(epoch) + " - " + str(accuracy) +"% Accuracy")
print("Best Accuracy for Perceptron model w/ first 10 words: " + str(best_accuracy) + "%")

: 

Perceptron model that's using the first 10 Word2Vec vectors is much better than Perceptron model using average vectors Word2Vec. This is what I thought would happen, as using average vectors will not capture any time-series aspect of the data. The average vector combines all the time-series data into a single vector, while the first 10 vectors still maintain the time-series that RNN will utilize to give better prediction. The accuracy for the first 10 vectors were sometimes 4% higher than that of average vector.

The only downside I see from using first 10 vectors compared to average vectors is that average vectors utilize the whole text, even when the text length is bigger than 10. 

# 5. Recurrent Neural Networks (30 points)

## a) RNN

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
valid = data_utils.TensorDataset(torch.Tensor(final_df["vector_review_20"]), torch.Tensor(e))
valid_loader = data_utils.DataLoader(valid, batch_size = 20, shuffle = True)

: 

In [ ]:
e = np.array(final_df["class"].astype('long')) - 1 
train = data_utils.TensorDataset(torch.Tensor(final_df["vector_review_20"]), torch.Tensor(e))
train_loader = data_utils.DataLoader(train, batch_size = 20, shuffle = True)

: 

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

: 

In [ ]:
model = MyRNN(300, 128, 3)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 10

best_accuracy = float('-inf')
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    model.eval() 
    correct = 0
    for data, target in valid_loader:
        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)

        e = torch.argmax(output, dim=1)
        correct += (e == target).float().sum()

    accuracy = correct.item()/len(valid_loader.dataset) * 100 
    
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'rnn.pt')
    print("Epoch " + str(epoch) + " - " + str(accuracy) +"% Accuracy")
print("Best Accuracy for RNN model: " + str(best_accuracy) + "%")

: 

## b) LSTM

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(MyLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        seq_len = x.size(0)
        w0 = torch.zeros(self.num_layers, seq_len, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, seq_len, self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (w0, c0))
        out = self.fc(out[:, -1, :])
        return out

: 

In [ ]:
model = MyLSTM(300, 128, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 20

best_accuracy = float('-inf')
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    model.eval() 
    correct = 0
    for data, target in valid_loader:
        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)

        e = torch.argmax(output, dim=1)
        correct += (e == target).float().sum()

    accuracy = correct.item()/len(valid_loader.dataset) * 100 
    
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'lstm.pt')
    print("Epoch " + str(epoch) + " - " + str(accuracy) +"% Accuracy")
print("Best Accuracy for LSTM model: " + str(best_accuracy) + "%")

: 

## c) GRU

In [ ]:
class MyGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(MyGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 10)

    def forward(self, x):
        seq_len = x.size(0)
        w0 = torch.zeros(self.num_layers, seq_len, self.hidden_size).to(x.device)
        out, _ = self.gru(x, w0)

        out = self.fc(out[:, -1, :])
        return out

: 

In [ ]:
model = MyGRU(300, 128, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

n_epochs = 20

best_accuracy = float('-inf')
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()

        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)
        
    model.eval() 
    correct = 0
    for data, target in valid_loader:
        output = model(data)
        target = target.to(dtype=torch.long)

        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)

        e = torch.argmax(output, dim=1)
        correct += (e == target).float().sum()

    accuracy = correct.item()/len(valid_loader.dataset) * 100 
    
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'gru.pt')
    print("Epoch " + str(epoch) + " - " + str(accuracy) +"% Accuracy")
print("Best Accuracy for GRU model: " + str(best_accuracy) + "%")

: 

GRU > LSTM > RNN. 

GRU/LSTM was better than RNN (sometimes I saw >7% accuracy better) while GRU was a bit better than LSTM sometimes. RNN in this case didn't seem to face any big vanishing gradient descent problem, as our given text wasn't as big (as it was shortend to 20 vectors). But even so, LSTM & GRU consistently had higher accuracy. 

Not only that, but I've noticed that GRU had faster training time. This might be since LSTM requires both w0 & c0, while GRU only requires w0. The lack of c0 for GRU might have also improved the accruracy, as due to smaller dataset that we used, since GRU had fewer parameters, that meant GRU model might have avoided overfitting. 

# Notes

In the folder will contain the Jupyter Notebook, the PDF file, and some of the saved models. Running the Jupyter Notebook will take some time (it took me 4 hours to run the whole thing), so some of the things that you can try if you want to run the notebook is...

1. Utilize final_df instead of training_df. Just get rid of anything to do with training_df. This will be the in the part of pre-processing and b) Custom Word2Vec. 
2. Don't run the GRU/LSTM models, but instead use the model that I saved

In the end, I was unable to get Word2Vec scores consistently higher than TF-IDF score for SVM & Perceptron. I noticed few other students had similar problem as I did, but I still think it's possible.